## Install and Import Requirements

In [1]:
!pip install -r requirements.txt
import os, time
from typing import List
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

     |████████████████████████████████| 78 kB 10.1 MB/s            
  Preparing metadata (setup.py) ... done
     |█████████████████████▉          | 313.0 MB 139.7 MB/s eta 0:00:02

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 458.3 MB 12 kB/s               
     |████████████████████████████████| 3.5 MB 95.6 MB/s            
     |████████████████████████████████| 462 kB 145.0 MB/s            
     |████████████████████████████████| 4.3 MB 85.8 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 42 kB 2.6 MB/s             
     |████████████████████████████████| 65 kB 757 kB/s              
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 1.3 MB 108.3 MB/s            
     |████████████████████████████████| 132 kB 143.5 MB/s            
     |████████████████████████████████| 5.6 MB 71.0 MB/s            
     |████████████████████████████████| 6.8 MB 66.1 MB/s            
     |████████████████████████████████| 67 kB 1.4 MB/s              
     |████████████████████████████████| 895 kB 69.9 MB/s            
     |████████████████████████████████| 781 kB 106.7 MB/s            
     |█████████

     |████████████████████████████████| 151 kB 135.6 MB/s            
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26432 sha256=d2312d552dc9a1ec14efb63dc83c6c77503c783ea3b12c0a7b33205dd32f9138
  Stored in directory: /home/ec2-user/.cache/pip/wheels/5f/63/c2/b85489bbea28cb5d36cfe197244f898428004fa3caa7a23116
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30878 sha256=aae06e6c3060adf9bd3b9dcef78e751aa57e4f6f41d499e9a28b03d7fd980582
  Stored in directory: /home/ec2-user/.cache/pip/wheels/22/4c/94/0583f60c9c5b6024ed64f290cb2d43b06bb4f75577dc3c93a7
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4850 sha256=c8482ff57093f8776f5ae183c990dcc10572ffd3d48f6849f3bfce955e0f1f25
  Stored in directory: /home/ec2-user/.cache/pip/wheels/93/2a/eb/e58dbcbc963549ee4f065ff80a59f274cc7210b6eab962acdc
Successfully built typing clang termcolor
  Attempting uninstall: typing-extensions
    Found existing installation: typing-exten

## Declare Globals and Configs

In [ ]:
DATASET_ENCODING = "ISO-8859-1"
INPUT_FEATURES_LOGGING = True
BATCH_SIZE = 8
EPOCHS = 1

# Tokenizer - preprocessing, prepares inputs for the model
TOKENIZER = BertTokenizer.from_pretrained("bert-base-uncased")

class Config:
    def __init__(self, model_file: str, csv_file: str, csv_size: int, dataset_columns: List[str], test_labels: List[int]):
        self.model_file = model_file
        self.csv_file = csv_file
        self.csv_size = csv_size
        self.dataset_columns = dataset_columns
        self.test_labels = test_labels

preprocessed_1600000_dataset_config = Config(
    "TRAINED_MODEL",
    "../Data/training.1600000.processed.noemoticon.csv",
    1600000,
    ["polarity", "ids", "date", "query", "user", "text"],
    [0, 2, 4]
)

vader_maxval_dataset_config = Config(
    "TRAINED_MODEL_VADER_MAXVAL",
    "../Data/classified_vader_scored_tweets_max_val.csv",
    30009,
    ["polarity", "text"],
    [-1, 0, 1])


vader_compval_dataset_config = Config(
    "TRAINED_MODEL_VADER_COMPVAL",
    "../Data/classified_vader_scored_tweets_compound_val.csv",
    30009,
    ["polarity", "text"],
    [-1, 0, 1])

manual_dataset_config = Config(
    "TRAINED_MODEL_MANUAL",
    "../Data/testdata.manual.2009.06.14.csv",
    500,
    ["polarity", "ids", "date", "query", "user", "text"],
    [0, 2, 4])

ian_dataset_config = Config(
    "TRAINED_MODEL_IAN",
    "../Data/ian.csv",
    250,
    ["polarity", "text"],
    [-1, 0, 1]
)

## Data Handling Functions

In [3]:
def get_csv_as_df(dataset_path: str, columns: List[str]):
    print("Open file:", dataset_path)
    return pd.read_csv(dataset_path, encoding =DATASET_ENCODING , names=columns)


def get_tf_dataset(input_examples):
    """
    Converts a dataframe of Input Examples
    :return: dataset (tensorflow object) of tensors
    """
    # What we are inputing into the vector
    features = []
    i = 0
    for e in list(input_examples):
        i += 1
        if i % 10000 == 0 and INPUT_FEATURES_LOGGING:
            print(f'{i} | {e}')
        input_dict = TOKENIZER.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=280,
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True,
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"], input_dict["token_type_ids"], input_dict['attention_mask'])
        features.append(InputFeatures(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label))

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    tf_dataset = tf.data.Dataset.from_generator(
        generator=gen,
        output_types=({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        output_shapes=(
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )
    return tf_dataset

## Load a Model

In [4]:
def load_model(model_file: str):
    location = os.path.realpath(os.path.join(os.getcwd(), os.path.dirname(__file__)))
    new_model = TFBertForSequenceClassification.from_pretrained(os.path.join(location, model_file), num_labels=3)
    new_model.summary()
    return new_model

## Model Testing Functions

In [5]:
def make_prediction(model, tweet: str, labels: List[int]):
    tf_batch = TOKENIZER(tweet, max_length=128, padding=True, truncation=True, return_tensors='tf')
    tf_outputs = model(tf_batch)
    tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
    tf_predictions = np.asarray(tf_predictions).tolist()
    if abs(tf_predictions[0][0] - tf_predictions[0][1]) < 0.00:
        return labels[1]
    elif tf_predictions[0][0] > tf_predictions[0][1]:
        return labels[0]
    else:
        return labels[2]
        

def test_model(model, config: Config):
    """
    model: object returned from load_model()
    config: Config for the file you are testing
    """
    df = get_csv_as_df(config.csv_file, config.dataset_columns)
    start_time = time.time()
    i = 0
    for index, row in df.iterrows():
        if index % 100 == 0:
            print(f"running {index}")
        if row['polarity'] == make_prediction(model, row['text'], config.test_labels):
            i += 1
    print(f"TESTING TOOK {(time.time() - start_time) / config.csv_size} per tweet")
    print(f"Accuracy: {i/config.csv_size}")

## Train a Model

In [6]:
def TRAIN(input_config: Config, output_config: Config):
    """
    input_config: pass None if starting from base model, otherwise pass config for model you want to fine tune
    output_config: Pass config for resulting model after training completes
    """

    print('\nLOADING DATA\n')
    df = get_csv_as_df(output_config.csv_file, output_config.dataset_columns)
    # remove unnecessary columns / cleanup text / map to our sentiment scores
    data_df = pd.DataFrame({
        'polarity': df['polarity'].apply(lambda x: x),
        'text': df['text'].replace(r'\n', '', regex=True)
    })

    # Converting data into proper input format for training the model
    # InputExamples are fed to the tokenizer. 
    input_examples = data_df.apply(lambda x: InputExample(None, text_a=x['text'], text_b=None, label=x['polarity']), axis=1)
    train_input_examples, validation_input_examples = train_test_split(input_examples, test_size=0.2)

    # Converting data to TensorFlow dataset objects
    ### "get_tf_dataset" IS THE ONLY PART I HAVENT CHECKED. JUST HOPING IT WORKS ###
    tf_train_dataset = get_tf_dataset(list(train_input_examples))
    tf_validation_dataset = get_tf_dataset(list(validation_input_examples))

    # Shuffle Data
    train_data = tf_train_dataset.shuffle(int(output_config.csv_size/10*0.8)).batch(BATCH_SIZE).repeat(2)
    validation_data = tf_validation_dataset.shuffle(int(output_config.csv_size/10*0.2)).batch(BATCH_SIZE).repeat(2)

    print('\nLOADING MODEL\n')
    if input_config == None:
        print('NO INPUT CONFIG PROVIDED, LOADING BASE MODEL')
        model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
    else:
        model = load_model(input_config.model_file)
        
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')]
    )
    
    print('\nTRAINING MODEL\n')
    model.fit(train_data, epochs=EPOCHS, validation_data=validation_data)

    print('\nSAVING MODEL\n')
    location = os.path.realpath(os.path.join(os.getcwd(), os.path.dirname(__file__)))
    path = os.path.join(location, output_config.model_file)
    model.save_pretrained(path, save_model=True)

## TRAINING

In [ ]:
TRAIN(None, vader_compval_dataset_config)


LOADING DATA

Open file: ../Data/classified_vader_scored_tweets_compound_val.csv


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


10000 | InputExample(guid=None, text_a='COVID-19 Relief Bill - DocumentCloud can anyone tell me how many times â\x80\x9cforeign assistanceâ\x80\x9d is stated in this 5593 page document?  https://t.co/uVTHY0vtom', text_b=None, label=1.0)
20000 | InputExample(guid=None, text_a='A special prosecutor is a step forward but we still need an independent commission to provide the American people with the whole truth. https://t.co/VcQAK1kaaI', text_b=None, label=2.0)

LOADING MODEL

NO INPUT CONFIG PROVIDED, LOADING BASE MODEL


Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



TRAINING MODEL

   3193/Unknown - 7125s 2s/step - loss: 0.4935 - accuracy: 0.7825

## EVALUATION